In [12]:
import pandas as pd
import os

In [2]:
project_dir = os.path.expanduser('~/disks/meso_S/data/stereo_prf/')
rootpath = os.path.join(project_dir,'sourcedata') # data directory
filepath = os.path.join(rootpath,'dcm2niix','job_list.csv')
print('Opening', filepath)

Opening /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/dcm2niix/job_list.csv


In [3]:
data = pd.read_csv(filepath, sep=';')

In [4]:
# check data integrity
print(data.head())

                                             sources  sub  ses data_type task  \
0  AM52/am52 pRF selected runs/01b_epi_retino_DIC...    1    2      func  prf   
1  AM52/am52 pRF selected runs/01b_epi_retino_DIC...    1    2      func  prf   
2  AM52/am52 pRF selected runs/01b_epi_retino_DIC...    1    2      func  prf   
3  AM52/am52 pRF selected runs/01b_epi_retino_DIC...    1    2      func  prf   
4  AM52/am52 pRF selected runs/01b_epi_retino_DIC...    1    2      func  prf   

   run modality  
0  1.0     bold  
1  2.0     bold  
2  3.0     bold  
3  4.0     bold  
4  5.0     bold  


In [5]:
sourceroot = os.path.join(rootpath,'Big_data_STAM')
destroot = os.path.join(rootpath,'dcm2niix')
overwrite = 0

In [6]:
print('dcm2niix does not work on jupyter notebook - we will print instead')
print('replace print from dcm2niix line with os.system in py batch code to make it work')

dcm2niix does not work on jupyter notebook - we will print instead
replace print from dcm2niix line with os.system in py batch code to make it work


In [49]:
# loop through the file to read the source and destination files, with mask renaming
# first create the folder structure
for i in range(0, len(data)):
    source = os.path.join(sourceroot,data.iloc[i].sources)
    if not(os.path.exists(source)):
        print('This source does not seem to exist - check it before we run the whole thing: '+source)
        break
    dest_dir_lvl1 = os.path.join(destroot,"sub-{p:02.0f}".format(p=data.iloc[i]['sub']))
    if not(os.path.exists(dest_dir_lvl1)):
        os.mkdir(dest_dir_lvl1)   
        print('Creating directory '+dest_dir_lvl1)
    dest_dir_lvl2 = os.path.join(dest_dir_lvl1,"ses-{p:02.0f}".format(p=data.iloc[i].ses))
    if not(os.path.exists(dest_dir_lvl2)):
        os.mkdir(dest_dir_lvl2)
        print('Creating directory '+dest_dir_lvl2)
    dest_dir_lvl3 = os.path.join(dest_dir_lvl2,data.iloc[i].data_type)    
    if not(os.path.exists(dest_dir_lvl3)):
        os.mkdir(dest_dir_lvl3)
        print('Creating directory '+dest_dir_lvl3)
print('Folder structure creation done')

Folder structure creation done


In [15]:
# then do the conversion job
for i in range(0, len(data)):
    source = os.path.join(sourceroot,data.iloc[i].sources)
    dest_dir = os.path.join(destroot,"sub-{p:02.0f}".format(p=data.iloc[i]['sub']),"ses-{p:02.0f}".format(p=data.iloc[i].ses),data.iloc[i].data_type)
    print(data.iloc[i].task)
    if pd.isnull(data.iloc[i].task)==0: # func case - task exists
        dest_file = "sub-{p:02.0f}".format(p=data.iloc[i]['sub'])+"_ses-{p:02.0f}".format(p=data.iloc[i].ses)+'_task-'+data.iloc[i].task+"_run-{p:1.0f}".format(p=data.iloc[i].run)+'_'+data.iloc[i].modality
    else:                              # anat case - no task, no run
        dest_file = "sub-{p:02.0f}".format(p=data.iloc[i]['sub'])+"_ses-{p:02.0f}".format(p=data.iloc[i].ses)+'_'+data.iloc[i].modality    
    dest_file_abs_path = os.path.join(dest_dir,dest_file+'.nii.gz')
    if os.path.exists(dest_file_abs_path) & (overwrite==0):
        print('File already exists - skip (from '+source+' toward '+dest_file_abs_path+')')
    else:
        if os.path.exists(dest_file_abs_path) & (overwrite==1):
            print('File already exists - overwrite '+dest_file)       
        print('Attempting to convert from '+source+' to '+dest_dir+' with file '+dest_file)
        print('dcm2niix -z y -s n -x n –b y –ba y -v 1 -f '+dest_file+' -o '+dest_dir+' \''+source+'\'') 
        # os.system('dcm2niix -z y -s n -x n –b y –ba y -v 1 -f \'test\' -o \'/home/achopin/disks/meso_S/data/stereo_prf/sourcedata/dcm2niix/sub-01/\' \'/home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM52/am52 pRF selected runs/01b_epi_retino_DICOM/epi01_neuro_retinotopy_11/\'')
        # -z y : gzip the files
        # -s n : convert all images in folder
        # -x n : do not crop
        # -b y : generate BIDS structure
        # -ba y: BIDS anonimization
        # -v 1 : verbose level 1
        # -f XXX: renaming mask here (or just output file name) 
        # -o YY : output folder YY
        # input folder XX ALWAYS needs to be the last argument
print('Niftii conversion done')

prf
File already exists - skip (from /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM52/am52 pRF selected runs/01b_epi_retino_DICOM/epi01_neuro_retinotopy_11/ toward sub-01_ses-02_task-prf_run-1_bold)
prf
File already exists - skip (from /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM52/am52 pRF selected runs/01b_epi_retino_DICOM/epi02_neuro_retinotopy_12/ toward sub-01_ses-02_task-prf_run-2_bold)
prf
File already exists - skip (from /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM52/am52 pRF selected runs/01b_epi_retino_DICOM/epi03_neuro_retinotopy_18/ toward sub-01_ses-02_task-prf_run-3_bold)
prf
File already exists - skip (from /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM52/am52 pRF selected runs/01b_epi_retino_DICOM/epi04_neuro_retinotopy_19/ toward sub-01_ses-02_task-prf_run-4_bold)
prf
File already exists - skip (from /home/achopin/disks/meso_S/data/stereo_prf/sourcedata/Big_data_STAM/AM5